# Time-MoE

https://github.com/Time-MoE/Time-MoE

In [1]:

!git clone https://github.com/Time-MoE/Time-MoE.git

Cloning into 'Time-MoE'...
remote: Enumerating objects: 175, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 175 (delta 43), reused 18 (delta 18), pack-reused 110 (from 1)
Receiving objects: 100% (175/175), 706.50 KiB | 3.66 MiB/s, done.
Resolving deltas: 100% (88/88), done.


In [2]:
%cd ./Time-MoE

/Users/cloudj/prj/wqu_capstone_git/Time-MoE


In [3]:
pip install -r requirements.txt

  Using cached PyYAML-6.0.2-cp310-cp310-macosx_11_0_arm64.whl.metadata (2.1 kB)
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 44.4 MB/s eta 0:00:0000:01
Using cached PyYAML-6.0.2-cp310-cp310-macosx_11_0_arm64.whl (171 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 34.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 31.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.2.0
    Uninstalling fsspec-2025.2.0:
      Successfully uninstalled fsspec-2025.2.0
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install flash-attn==2.6.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 32.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      fatal: not a git repository (or any of the parent directories): .git
      
      
      torch.__version__  = 2.6.0
      
      
      /private/var/folders/vy/swc92n890xq1d7g08y98sqwr0000gn/T/pip-install-t4jfvvpz/flash-attn_7c93ad888d454f0e8e97df0778a7d321/setup.py:95: UserWarning: flash_attn was requested, but nvcc was not found.  Are you sure your environment has nvcc available?  If you're installing within a container from https://hub.docker.com/r/pytorch/pytorch, only images whose names contain 'devel' will provide nvcc.
        warnings.warn(
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/private/var/fold

Forecasting Sample (from https://github.com/Time-MoE/Time-MoE)

In [16]:
import numpy as np
import torch
from transformers import AutoModelForCausalLM

# context_length = 12
# seqs = torch.randn(2, context_length)  # tensor shape is [batch_size, context_length]
# seqs = np.array([
#     1., 1.1, 1.2, 1.1, 1.0, 0.9, 0.8, 0.7, 0.7, 0.8, 0.9, 1., 1.1, 1.2
# ])
seqs = [
    1., 1.1, 1.2, 1.1, 1.0, 0.9, 0.8, 0.7, 0.7, 0.8, 0.9, 1., 1.1, 1.2
]
prediction_length = 2
context_length = len(seqs) - prediction_length 
seqs = torch.Tensor([seqs[0:context_length]])

model = AutoModelForCausalLM.from_pretrained(
    'Maple728/TimeMoE-50M',
    device_map="cpu",  # use "cpu" for CPU inference, and "cuda" for GPU inference.
    trust_remote_code=True,
)

# use it when the flash-attn is available
# model = AutoModelForCausalLM.from_pretrained('Maple728/TimeMoE-50M', device_map="auto", attn_implementation='flash_attention_2', trust_remote_code=True)

# normalize seqs
mean, std = seqs.mean(dim=-1, keepdim=True), seqs.std(dim=-1, keepdim=True)
normed_seqs = (seqs - mean) / std

# forecast
# prediction_length = 6
output = model.generate(normed_seqs, max_new_tokens=prediction_length)  # shape is [batch_size, 12 + 6]
normed_predictions = output[:, -prediction_length:]  # shape is [batch_size, 6]

# inverse normalize
predictions = normed_predictions * std + mean

print(predictions)

/Users/cloudj/opt/anaconda3/envs/tabe/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/cloudj/opt/anaconda3/envs/tabe/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


KeyboardInterrupt: 

In [9]:
!python run_eval.py -d ../dataset/ETT-small/ETTh1.csv -p 96

7062.20s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


/Users/cloudj/opt/anaconda3/envs/tabe/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2025-02-20 14:40:14,677 - log_util.py[pid:31365;line:52:log_in_local_rank_0] - INFO: >>> Split test data from 2017/10/2 16:00 to 2018/2/20 23:00, and evaluation start date is: 2017/10/24 00:00
 10%|███▊                                 | 62/609 [1:09:18<12:01:19, 79.12s/it]^C
